In [4]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from collections import Counter

In [101]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/pawel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/pawel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Załadowanie plików z danymi

Pliki z danymi znajduję się w folderze `./data/`:

In [7]:
!ls ./data

test.tsv  train.tsv  valid.tsv


In [55]:
train_data = pd.read_csv('./data/train.tsv', sep='\t')

In [9]:
train_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


W danych walidacyjnych i testowych brakuje nagłówków, więc dodaję je z pliku treningowego.

In [10]:
valid_data = pd.read_csv('./data/valid.tsv', sep='\t', header=None, names=train_data.columns)

In [11]:
valid_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,140461,7622,democracy,2
1,140462,7622,and civic action laudable,3
2,140463,7622,civic action laudable,2
3,140464,7622,action laudable,3
4,140465,7623,Griffin & Co. manage to be spectacularly outra...,4


In [12]:
valid_data.tail()

,PhraseId,SentenceId,Phrase,Sentiment
7795,148256,8068,a storm,2
7796,148257,8068,as a fringe feminist conspiracy theorist,1
7797,148258,8068,a fringe feminist conspiracy theorist,2
7798,148259,8068,fringe feminist conspiracy theorist,1
7799,148260,8068,fringe,2


In [13]:
test_data = pd.read_csv('./data/test.tsv', sep='\t', header=None, names=train_data.columns)

In [14]:
test_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,148261,8068,feminist conspiracy theorist,2
1,148262,8068,conspiracy theorist,2
2,148263,8068,theorist,2
3,148264,8068,named Dirty Dick,2
4,148265,8068,Dirty Dick,2


In [15]:
test_data.tail()

,PhraseId,SentenceId,Phrase,Sentiment
7795,156056,8544,Hearst 's,2
7796,156057,8544,forced avuncular chortles,1
7797,156058,8544,avuncular chortles,3
7798,156059,8544,avuncular,2
7799,156060,8544,chortles,2


Sprawdzam czy istnieją komórki z wartością `NaN`:

In [16]:
list(map(lambda x: x.isnull().any().any(), (train_data, valid_data, test_data)))

[False, False, False]

# Wstępna analiza danych

## Forma i wygląd danych

Zdania oznaczone przez `SentenceId` są podzielone na frazy (`Phrase`), z których każda posiada określony `Sentiment`:

In [17]:
train_data.loc[train_data['SentenceId'] == 42]

,PhraseId,SentenceId,Phrase,Sentiment
1134,1135,42,Vincent Gallo is right at home in this French ...,3
1135,1136,42,Vincent Gallo,2
1136,1137,42,Vincent,2
1137,1138,42,Gallo,2
1138,1139,42,is right at home in this French shocker playin...,3
1139,1140,42,is right at home in this French shocker playin...,3
1140,1141,42,is right at home in this French shocker,2
1141,1142,42,right at home in this French shocker,3
1142,1143,42,right at home,3
1143,1144,42,right,3


Powyższa ramka danych uświadamia, że w skład `Phrase` wchodzą zarówno jednowyrazowe napisy oraz napisy składające się z wielu wyrazów. Warto zauważyć, że niektóre `Phrase` składają się tylko z tzw. *stopwords*. Okazuje się, że frazy będące stopwords to nieznaczna część danych, więc w dalszych krokach będę używał metody wykluczania stopwords:

In [18]:
train_data['Phrase'].apply(lambda x: x in stopwords.words('english')).value_counts(normalize=True)

False    0.999179
True     0.000821
Name: Phrase, dtype: float64

Czy we frazach występują znaki interpunkcyjne:

In [19]:
train_data['Phrase'].apply(lambda x: any([punc in x for punc in string.punctuation])).value_counts(normalize=True)

False    0.573704
True     0.426296
Name: Phrase, dtype: float64

### Rozkład zmiennej zależnej według fraz

Sprawdzam jaki jest rozkład zmiennej zależnej (`Sentiment`) według fraz składających się na zdania (`PhraseId`). Przeważa klasa środkowa, czyli neutralna. Rozkład podobny dla wszystkich trzech zestawów danych. Interesujące jest to, że frazy oznaczone jako skrajne(`0` oraz `4`) są średnio najdłuższe:

In [20]:
def get_by_phrase_stats(set_df):
    no_rows = set_df.shape[0]

    aggregation = {
        'Phrase': {
            'count': 'count',
            'freq': lambda x: x.shape[0]/no_rows,
            'avg_number_of_words': lambda x: np.mean([len(phrase.split()) for phrase in x])
        }    
    }

    return set_df.groupby('Sentiment').agg(aggregation)

In [21]:
list(map(lambda x: get_by_phrase_stats(x), (train_data, valid_data, test_data)))

/home/pawel/.local/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


[          Phrase                              
            count      freq avg_number_of_words
 Sentiment                                     
 0           6318  0.045129           12.035296
 1          24081  0.172008            9.076118
 2          71738  0.512418            5.139466
 3          29602  0.211444            8.404229
 4           8260  0.059000           10.662228,
           Phrase                              
            count      freq avg_number_of_words
 Sentiment                                     
 0            328  0.042051           13.435976
 1           1519  0.194744            9.903226
 2           3788  0.485641            5.714361
 3           1696  0.217436            8.825472
 4            469  0.060128           11.285714,
           Phrase                              
            count      freq avg_number_of_words
 Sentiment                                     
 0            417  0.053462           12.177458
 1           1605  0.205769           

# Zdefiniowane zadania

W dalszych krokach będę próbował określić wartość kolumny `Sentiment` dla każdej z `Phrase`.

# Podejście z pełnymi zdaniami - ustalenie benchmarku

W tym podejściu skupiam się jedynie na pełnych zdaniach. Innymi słowy ignoruję podział zdania na frazy i jedynie na podstawie całego zdania próbuję określić wartość `Sentiment`. Dla fraz wchodzących w skład danego zdania przypiszę wartości `Sentiment` uzyskane dla tego zdania. Ze względu na niedoskonałość tego podejścia będę mógł je potraktować jako swego rodzaju benchmark.

W zestawie treningowym i walidacyjnym zostawiam tylko pełne zdania:

In [22]:
train_data_full_sent_only = train_data.drop_duplicates(subset='SentenceId')

In [23]:
train_data_full_sent_only.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1


In [24]:
valid_data_full_sent_only = valid_data.drop_duplicates(subset='SentenceId')

In [25]:
valid_data_full_sent_only.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,140461,7622,democracy,2
4,140465,7623,Griffin & Co. manage to be spectacularly outra...,4
14,140475,7624,Like The English Patient and The Unbearable Li...,3
49,140510,7625,`` Auto Focus '' works as an unusual biopic an...,3
68,140529,7626,"Very amusing , not the usual route in a thrill...",3


Poniższa funkcja sprowadza dane zdanie do mniej złożonej reprezentacji (nadal literowej):

In [26]:
stemmer = WordNetLemmatizer()

def simplify_sentence(sentence):
    # usuwanie znaków specjalnych (wszystkich poza alfanumerycznymi)
    sentence = re.sub(r'\W', ' ', sentence)
    
    # usuwanie wszystkich pojedynczych liter
    sentence = re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', sentence)

    # zamiana na małe litery
    sentence = sentence.lower()
    
    # wyciągnięcie pojedynczych słów
    sentence = sentence.split()
    
    # sprowadzenie słów do form podstawowych (lematów)
    # uwaga: sentence to teraz lista
    sentence = [stemmer.lemmatize(word) for word in sentence]
    
    # powrót do str
    return " ".join(sentence)

In [27]:
train_data_full_sent_only['Phrase_cleaned'] = train_data_full_sent_only['Phrase'].apply(lambda x: simplify_sentence(x))

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
valid_data_full_sent_only['Phrase_cleaned'] = valid_data_full_sent_only['Phrase'].apply(lambda x: simplify_sentence(x))

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Zamieniam uproszczone (wyczyszczone zdania) na reprezentację bag of words. 

`max_features`: zwracam uwagę tylko na 1000 najczęściej występujących słów, 

`min_df` to liczba minimalna liczba zdań, w których musi wystąpić słowo, żeby nie zostać zignorowanym, 

`max_df` to maksymalny odsetek wszystkich zdań, w których słowo może wystąpić - służy do zignorowania słów, które występują w prawie każdym zdaniu

`stop_words` to słowa, które występują na tyle często w języku, że nie wnoszą żadnych informacji w zadaniu klasyfikacji

In [29]:
vectorizer = CountVectorizer(max_features=30, min_df=5, max_df=0.75, stop_words=stopwords.words('english'))  

In [30]:
train_data_vectorized = vectorizer.fit_transform(train_data_full_sent_only['Phrase_cleaned'])

In [31]:
valid_data_vectorized = vectorizer.fit_transform(valid_data_full_sent_only['Phrase_cleaned'])

Dotychczasowa reprezentacja wektorowa zdań mimo że uwzględnia fakt bardzo rzadkich słów (`min_df`) oraz słów występujących w prawie każdym zdaniu (`max_df`) może zostać ulepszona podejściem TFIDF. Podejście to nadaje większą wagę słowom bardziej informatywnym, tzn. tym które występują często w małej ilości zdań.

In [32]:
tfidfconverter = TfidfTransformer()  
train_data_tfidf = tfidfconverter.fit_transform(train_data_vectorized).toarray() 

In [33]:
valid_data_tfidf = tfidfconverter.fit_transform(valid_data_vectorized).toarray() 

Używam lasu losowego z 500 drzewami w celu dokonania klasyfikacji:

In [34]:
rf_classifier = RandomForestClassifier(n_estimators=500, random_state=0) 

In [35]:
rf_classifier.fit(train_data_tfidf, train_data_full_sent_only['Sentiment'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

Przygotowuję dane testowe, na podstawie których dokonam predykcji:

In [36]:
test_data_full_sent_only = test_data.drop_duplicates(subset='SentenceId')

In [37]:
test_data_full_sent_only['Phrase_cleaned'] = test_data_full_sent_only['Phrase'].apply(lambda x: simplify_sentence(x))

/home/pawel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
test_data_vectorize = vectorizer.fit_transform(test_data_full_sent_only['Phrase_cleaned'])

In [39]:
test_data_tfidf = tfidfconverter.fit_transform(test_data_vectorize).toarray()

In [40]:
y_pred = rf_classifier.predict(test_data_tfidf)

`y_pred` muszą zostać skopiowane do odpowiadających `PhraseId`, aby można było obliczyć dokładność klasyfikatora. Z jednej strony mam tyle unikalnych `SentenceId` w `test_data`:

In [41]:
len(y_pred)

475

... z drugiej strony w `test_data` jest tyle unikalnych `PhraseId`:

In [42]:
test_data.shape[0]

7800

Wykonuję złączenie dwóch ramek danych po `SentenceId`:

In [43]:
sent_only_preds = pd.DataFrame(data={"predicted_class": y_pred, "SentenceId": test_data_full_sent_only['SentenceId']}, index=test_data_full_sent_only.index)

In [44]:
sent_only_preds.head()

,predicted_class,SentenceId
0,3,8068
5,1,8069
23,3,8070
61,4,8071
71,1,8072


In [45]:
test_data_w_predicted = pd.merge(test_data, sent_only_preds, how='left', on='SentenceId')
test_data_w_predicted.head()

,PhraseId,SentenceId,Phrase,Sentiment,predicted_class
0,148261,8068,feminist conspiracy theorist,2,3
1,148262,8068,conspiracy theorist,2,3
2,148263,8068,theorist,2,3
3,148264,8068,named Dirty Dick,2,3
4,148265,8068,Dirty Dick,2,3


Biorąc pod uwagę tragiczne wyniki klasyfikatora za benchmark w dalszych rozważaniach będę uznawał częstość występowania klasy dominującej w zadaniach:

In [46]:
print(accuracy_score(test_data_w_predicted['Sentiment'], test_data_w_predicted['predicted_class']))
print("Benchmark dla accuracy będzie wynosił: ", max(get_by_phrase_stats(test_data)['Phrase']['freq']))

0.20717948717948717
Benchmark dla accuracy będzie wynosił:  0.49


/home/pawel/.local/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


# Modelowanie embeddingsów

W dalszej części analizy będę posługiwał się podejściem z uczenia głębokiego. W przypadku modelowania tekstu potrzebne jest utworzenie wektorów zanurzonych. Na początku spróbuję utworzyć je własnoręcznie, później skorzystam z już obliczonych, publicznie dostępnych.

## Trenowanie własnych embeddingsów

Poniżej korzystam z podejścia word2vec. Jako, że mam pięć kategorii w `Sentiment` to utworzę embeddingi dla 5 przypadków.

Funkcja czyszcząca zdania. Mimo wcześniejszego zdefiniowania funkcji służącej wyczyszczeniu tekstu tutaj korzystam z funkcji podanej w dedykowanym word2vec notebook'owi z zajęć:

In [45]:
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

Funkcja dodająca słowa do słownika:

In [46]:
def add_doc_to_vocab(sent, vocab):
    tokens = clean_doc(sent)
    vocab.update(tokens)

In [47]:
vocabs = dict()

for sentiment_value in train_data.Sentiment.unique():
    tmp_vocab = Counter()
    tmp_df = train_data.loc[train_data['Sentiment']==sentiment_value]
    for phrase in tmp_df['Phrase']:
        add_doc_to_vocab(phrase, tmp_vocab)
    vocabs["sentiment_{}".format(sentiment_value)] = tmp_vocab

In [48]:
for key in vocabs.keys():
    print(key)

sentiment_1
sentiment_2
sentiment_3
sentiment_4
sentiment_0


Sprawdzam najczęstsze słowa dla każdej wartości `Sentiment`:

In [49]:
for senti_value, cnt in vocabs.items():
    print("Most common ", senti_value, cnt.most_common(10))
    print("Least common ", senti_value, cnt.most_common()[-10:])

Most common  sentiment_1 [('nt', 1368), ('movie', 1216), ('film', 1078), ('The', 925), ('like', 739), ('one', 578), ('much', 526), ('RRB', 447), ('little', 429), ('story', 409)]
Least common  sentiment_1 [('JFK', 1), ('nuts', 1), ('soulsearching', 1), ('deliberateness', 1), ('deny', 1), ('seriousness', 1), ('flatout', 1), ('pleased', 1), ('matineestyle', 1), ('restrictive', 1)]
Most common  sentiment_2 [('film', 1919), ('movie', 1652), ('The', 1635), ('nt', 1131), ('one', 1085), ('RRB', 989), ('like', 926), ('story', 808), ('LRB', 780), ('much', 608)]
Least common  sentiment_2 [('transfixes', 1), ('freakout', 1), ('Songs', 1), ('During', 1), ('boorishness', 1), ('Oscarworthy', 1), ('scorcher', 1), ('fearlessly', 1), ('deny', 1), ('bangup', 1)]
Most common  sentiment_3 [('film', 1619), ('movie', 1147), ('The', 945), ('good', 768), ('one', 698), ('story', 603), ('funny', 579), ('nt', 517), ('RRB', 494), ('like', 464)]
Least common  sentiment_3 [('artconscious', 1), ('Accidental', 1), ('E

Ze słownika usunę te słowa, które wystąpiły tylko raz. Najpierw tworzę słownik dla całego zestawu treningowe łącząc słownika dla poszczególnych wartości `Sentiment`:

In [50]:
complete_vocab = sum(list(vocabs.values()), Counter())

In [51]:
complete_vocab.most_common(5)

[('film', 5896), ('movie', 5179), ('The', 4139), ('nt', 3520), ('one', 2993)]

In [52]:
min_occurence = 2
tokens_more_than_once = [k for k, v in complete_vocab.items() if v >= min_occurence]
vocab_more_than_once = set(tokens_more_than_once) #dostęp O(1)

Kolejnym krokiem jest przefiltrowanie danych treningowych pod kątem słów, które zebrałem w słowniku. Definiuję funkcję, która poza podstawowym czyszczeniem sprawdza także czy słowo należy do słów ze słownika:

In [53]:
def clean_doc_with_vocab(phrase, vocab):
    # split into tokens by white space
    tokens = phrase.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab] # sprawdzamy czy tokeny występują w słowniku
    tokens = ' '.join(tokens)
    return tokens

In [54]:
train_docs = []
for phrase in train_data['Phrase']:
    res = clean_doc_with_vocab(phrase, vocab_more_than_once)
    if res != '':
        train_docs.append(res)

In [55]:
train_docs[:10]

['series escapades demonstrating adage good goose also good gander occasionally amuses none amounts much story',
 'series escapades demonstrating adage good goose',
 'series',
 'series',
 'escapades demonstrating adage good goose',
 'escapades demonstrating adage good goose',
 'escapades',
 'demonstrating adage good goose',
 'demonstrating adage',
 'demonstrating']

Tworzę tokenizer, który zmapuje słowa na liczby:

In [60]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_docs)
encoded_docs = tokenizer.texts_to_sequences(train_docs)

Sprawdzam jak wygląda `tokenizer`:

In [67]:
tokenizer.word_index['the']

3

Zmieniła się struktura danych, które do tej pory były ciągem słów i spacji:

In [68]:
print(train_docs[51])

quiet introspective entertaining independent


Teraz struktura to ciąg liczb oddzielonych przecinkiem:

In [69]:
print(encoded_docs[51])

[661, 4457, 99, 3285]


Sieć neuronowa, której produktem ubocznym są embeddingi wymaga, aby wejście było stałej długości. Biorę najdłuższy element i wyrównuję pozostałe elementy do jego długości, wstawiając w puste miejsca `0` (puste miejsca występują po właściwej zawartości):

In [70]:
max_len = max([len(s.split()) for s in train_docs])

Najdłuższy ciąg słów ma długość:

In [72]:
max_len

30

In [74]:
train_X = pad_sequences(encoded_docs, maxlen=max_len, padding='post')

Okazuje się, że wymiar `encoded_docs` nie zgadza się z wymiarem pierwotnej ramki danych `train_data`, co utrudnia dodanie zmiennej zależnej. Sytuacja spowodowana jest tym, że niektóre obserwacje z `train_data` zawierały same stopwords.

In [75]:
len(train_X)

138616

In [76]:
train_data.shape

(139999, 4)

Poddaję ramkę danych `train_data` (kolumnę `Phrase`) działaniu funkcji `clean_doc()` a następnie usuwam wiersze, gdzie `Phrase` jest pustym ciągiem znaków:

In [100]:
train_data_clean_phrase = train_data.copy(deep=True)
train_data_clean_phrase['Phrase'] = train_data['Phrase'].apply(lambda x: " ".join(clean_doc(x)))

Widać, że niektóre `Phrase` są teraz puste:

In [101]:
train_data_clean_phrase.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,series escapades demonstrating adage good goos...,1
1,2,1,series escapades demonstrating adage good goose,2
2,3,1,series,2
3,4,1,,2
4,5,1,series,2


In [102]:
train_data_clean_phrase.shape[0]

139999

Usuwam puste `Phrase`:

In [103]:
train_data_clean_phrase = train_data_clean_phrase.drop(train_data_clean_phrase[train_data_clean_phrase.Phrase == ""].index)

In [104]:
train_data_clean_phrase.shape[0]

138650

Rozmiar ciągle się nie zgadza, ponieważ użyłem funkcji `clean_doc_with_vocab()`.

In [105]:
train_data_clean_phrase['Phrase'] = train_data_clean_phrase['Phrase'].apply(lambda x: clean_doc_with_vocab(x, vocab_more_than_once))

Usuwam puste wiersze:

In [110]:
train_data_clean_phrase = train_data_clean_phrase.drop(train_data_clean_phrase[train_data_clean_phrase.Phrase == ""].index)

In [111]:
train_data_clean_phrase.shape[0]

138616

Teraz mogę bezpiecznie przekopiować `Sentiment` z ramki danych.

In [113]:
train_y = np.array(train_data_clean_phrase['Sentiment'])

Czynności dotyczące obróbki tekstu i zamiany na liczby powtarzam dla zestawu testowego. Tym razem tekst obrabiam wewnątrz ramki danych, żeby nie stracić informacji, które wiersze zostały z niej usunięte:

In [115]:
test_data_clean_phrase = test_data.copy(deep=True)
test_data_clean_phrase['Phrase'] = test_data_clean_phrase['Phrase'].apply(lambda x: " ".join(clean_doc(x)))

In [117]:
test_data_clean_phrase['Phrase'] = test_data_clean_phrase['Phrase'].apply(lambda x: clean_doc_with_vocab(x, vocab_more_than_once))

In [119]:
test_data_clean_phrase = test_data_clean_phrase.drop(test_data_clean_phrase[test_data_clean_phrase.Phrase == ""].index)

In [120]:
test_data_clean_phrase.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,148261,8068,feminist conspiracy,2
1,148262,8068,conspiracy,2
3,148264,8068,named Dirty Dick,2
4,148265,8068,Dirty Dick,2
5,148266,8069,The action XXX blast excitement,4


In [124]:
test_docs = test_data_clean_phrase['Phrase'].tolist()

In [125]:
test_docs[:5]

['feminist conspiracy',
 'conspiracy',
 'named Dirty Dick',
 'Dirty Dick',
 'The action XXX blast excitement']

In [130]:
encoded_test_docs = tokenizer.texts_to_sequences(test_docs) # słowa jako liczby
test_X = pad_sequences(encoded_docs, maxlen=max_len, padding='post') # stała długość wejścia
test_y = np.array(test_data_clean_phrase['Sentiment']) # label danych testowych

Potrzebny jest rozmiar słownika. Pamiętam o zostawieniu oddzielnego znaku dla paddingu wykorzystanego w wyrównywaniu do stałej długości wejścia:

In [131]:
v_size = len(tokenizer.word_index) + 1
v_size

15167

W celu stworzenia embeddingsów korzystam z sieci z zajęć:

In [149]:
model = Sequential()
model.add(Embedding(v_size, 20, input_length=max_len)) # 100-wymiarowy embedding - 
                                                               # wektor zanurzony o długości 100
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=10, verbose=2)
# loss, acc = model.evaluate(test_X, test_y, verbose=0)
loss, acc = model.evaluate(train_X, train_y, verbose=0) #sprawdzam czy loss zmienia się na danych treningowych - powinno przechodzić do 1
print('Test Accuracy: %f' % (acc*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 30, 20)            303340    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 23, 32)            5152      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 11, 32)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 352)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                3530      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 11        
Total params: 312,033
Trainable params: 312,033
Non-trainable params: 0
_________________________________________________________________
None

In [146]:
train_X

array([[  216, 13938,  5295, ...,     0,     0,     0],
       [  216, 13938,  5295, ...,     0,     0,     0],
       [  216,     0,     0, ...,     0,     0,     0],
       ...,
       [  183,   339,  3050, ...,     0,     0,     0],
       [    5,   379,     2, ...,     0,     0,     0],
       [    5,   379,     2, ...,     0,     0,     0]], dtype=int32)

<hr>

# Embeggings - 24.05

## Czyszczenie danych

Poniżej zdefiniowana funkcja jest zbyt agresywna, ponieważ dla niektórych obserwacji w `Phrase` (np. tych zawierających same *stopwords*) będzie zwracała puste napisy. W jej miejsce stosuję wyrażenie regularne, które usuwa wszystkie znaki poza literami łacińskimi (małymi i wielkimi), cyframi oraz odstępami (np. spacje, taby, nowe linie). Mimo, że wykorzystany w dalszej części obiekt `Tokenizer` filtruje po znakach m.in. znakach odstępu (`\s`) usuwając je, to na razie muszę zostawić odstępy, które stanowią granicę wyrazów (tokenów) w zdaniu.

In [47]:
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return " ".join(tokens)

In [56]:
train_data['Phrase'] = train_data['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))

In [57]:
train_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [58]:
test_data['Phrase'] = test_data['Phrase'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))

In [59]:
test_data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,148261,8068,feminist conspiracy theorist,2
1,148262,8068,conspiracy theorist,2
2,148263,8068,theorist,2
3,148264,8068,named Dirty Dick,2
4,148265,8068,Dirty Dick,2


Na wejściu do sieci musi pojawiać się wektor stałej długości. Dodatkowym wymaganiem jest to, żeby słowa w nim zawarte były zakodowane jako liczby.

In [60]:
X_tr = train_data['Phrase']
y_tr = train_data['Sentiment']

Definiuję `Tokenizer`, którego jednym z argumentów jest m.in. to ile słów z wejścia zachowa. Domyślne ustawienia (`num_words`) to `num_words-1`, co oznacza, że słowa o najniższej częstości występowania nie przejdą etapu tokenizacji. Wydaje się to rozsądnym pomysłem, ponieważ słowa najrzadziej występujące to zazwyczaj te występujące tylko raz. Dodatkowo przerabiam wszystkie znaki na wersje pisane małą literą (argument `lower=True`).

In [65]:
tokenizer = Tokenizer()

W tym kroku tworzę słownik, który ma następującą postać `słownik['the']=1`, czyli każde słowo z zestawu treningowego jest kodowane jako liczba.

In [66]:
tokenizer.fit_on_texts(X_tr.values)

In [73]:
X_ts = test_data['Phrase']

Poniżej elementy zdania zamieniane są na liczby:

In [74]:
X_tr_tokenized = tokenizer.texts_to_sequences(X_tr)
X_ts_tokenized = tokenizer.texts_to_sequences(X_ts)

Poniższy przykład pokazuje, że zdania zostały zamienione na listy długości równej liczbie tokenów w pierwotnych (ale oczyszczonych regexem) zdaniach:

In [78]:
X_tr[55], X_tr_tokenized[55]

('to much of a story', [5, 54, 3, 2, 40])

In [81]:
X_ts[22], X_ts_tokenized[22]

('for excitement', [13, 1472])

Należy wyrównać listy do tej samej długości (długości najliczniejszego pod względem tokenów zdania). Argument `padding='post'` wskazuje, że w krótszych niż `max_len` zdaniach zera będą dodane na końcu:

In [85]:
max_len = max([len(x.split()) for x in train_data['Phrase']])
max_len

48

In [87]:
X_tr_tokenized = pad_sequences(X_tr_tokenized, max_len, padding='post')

In [88]:
X_tr_tokenized

array([[   2,  316,    3, ...,    0,    0,    0],
       [   2,  316,    3, ...,    0,    0,    0],
       [   2,  316,    0, ...,    0,    0,    0],
       ...,
       [ 280,  442, 3148, ...,    0,    0,    0],
       [   8,   28,  482, ...,    0,    0,    0],
       [  28,  482,   17, ...,    0,    0,    0]], dtype=int32)

In [89]:
X_ts_tokenized = pad_sequences(X_ts_tokenized, max_len, padding='post')

Długość tokenizowanych zdań w obu zestawach jest równa:

In [91]:
X_tr_tokenized.shape[1] == X_ts_tokenized.shape[1]

True

### Budowa sieci

Każdy z wektorów zanurzonych w sieci będzie miał wymiar 100. Zgodnie z dokumentacją `input_dim` dla w `Embedding()` musi być równy liczbie słów plus 1 (plus 1, aby ująć `0` za pomocą, którego wypełnia się wektory krótsze niż maksymalny wymiar). `input_length` to argument informujący o długości wejścia.

In [109]:
embeddings_dim = 100
input_dim = len(tokenizer.word_index)+1
model = Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=embeddings_dim, input_length=max_len))

Warstwa Long Short Term Memory (`LSTM`) oraz warstawa klasyfikująca `Dense`, w której umieszczam 5 neuronów (jedna dla każdej z przewidywanych klas).

In [110]:
model.add(LSTM(units=128))
model.add(Dense(units=1, activation='softmax'))

Warto zwrócić uwagę na funkcję straty `'sparse_categorical_crossentropy'`, która powinna być używana w przypadku, gdy klasy są rozłączne. W naszym przypadku oznacza to, że ostatnia warstwa nie zwraca prawdopodobieństw należenia do różnych klas. Wspomniana funkcja jest używana, gdy zmienna zależna nie jest kodowana w postaci one-hot, lecz jako liczby całkowite (np. 1, 2, 3, 4, 5).

In [111]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'acc'])

In [112]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 48, 100)           1540600   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,657,977
Trainable params: 1,657,977
Non-trainable params: 0
_________________________________________________________________


In [113]:
model.fit(X_tr_tokenized, y_tr, epochs = 2, verbose = 1)

/home/pawel/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
139999/139999 [==============================] - 207s 1ms/step - loss: 1.9343 - mean_absolute_error: 1.1574 - acc: 0.1720
Epoch 2/2
139999/139999 [==============================] - 221s 2ms/step - loss: 1.9343 - mean_absolute_error: 1.1574 - acc: 0.1720


## Notatki
1. Wykorzystać FastText crawl 300d 2M.